# Abordagem PAD no ambiente Python para um caso de teste de computação científica

Exemplo mostrando abordagen de Processamento de Alto Desempenho (PAD ou HPC) no ambiente Python, para a solução de um caso de teste de computação científica, usando o supercomputador Santos Dumont (SD). Para este exemplo foi selecionado o compilador Numba rodando em CPU e em GPU, usando os nós B710 e Sequana.

## Caso de teste: estêncil de cinco pontos

(versão sequencial)

O caso de teste adotado é um conhecido problema de transferência de calor sobre uma superfície finita, modelado pela equação diferencial parcial de Poisson, que modela a distribuição de temperatura normalizada sobre a superfície ao longo de uma série de iterações que compõem a simulação. A equação de Poisson discretizada 2D com um estêncil de 5 pontos pode ser expressa por:

$$
\frac{\partial^2 U}{\partial x^2} +
\frac{\partial^2 U}{\partial y^2} \approx
\frac{U_{i+1,j}+U_{i,j+1}-4U_{i,j}+U_{i-1,j}+U_{i,j-1}}{h^2}
$$

Comumente empregada para soluções numéricas, esta equação é discretizada em uma grade finita e resolvida por um método de diferenças finitas. O algoritmo específico requer o cálculo de um estêncil de 5 pontos sobre a grade de domínio 2D para atualizar as temperaturas a cada etapa de tempo. Para este problema, são assumidos um campo de temperatura uniforme inicial com valor zero sobre a superfície, e condições adiabáticas ou de contorno de Dirichlet. O estêncil de 5 pontos consiste em atualizar um ponto da grade fazendo a média das temperaturas de si mesmo com as temperaturas de seus quatro vizinhos, esquerda-direita e cima-baixo na grade. O campo de temperatura $ U $ é definido sobre uma grade discreta $ (x, y) $ com resoluções espaciais $ \Delta x = \Delta y = h $. Assim, a discretização mapeia as coordenadas cartesianas reais $ (x, y) $ para uma grade discreta $ (i, j) $, com $ U_ {x, y} = U_ {i, j} \,, \, U_ {x + h, y} = U_ {i + 1, j} \, $ para a dimensão $ x $, e analogamente para a dimensão $ y $.

![](img/grade.png)

Três fontes de calor de taxa constante foram colocadas em pontos de grade localizados, e cada uma introduz uma quantidade de calor unitária a cada passo de tempo. A simulação de transferência de calor é modelada por um número finito de etapas de tempo, sendo todos os pontos da grade atualizados a cada etapa. A distribuição da temperatura será determinada pelas fontes de calor e pelas condições de contorno de Dirichlet, o que implica em temperatura zero nos pontos de fronteira da grade.

![](img/fontes.png)

---

# Implementação

A parte computacionalmente intensiva é a atualização dos pontos da grade (atualização da matriz) utilizando a equação (cálculo do estêncil), onde dois laços de repetição são utilizados, um para cada dimensão.

Em cada iteração, após a atualização da grade uma unidade de calor é inserida em três pontos de inserção de calor.

Duas matrizes são utilizadas alternadamente nas iterações pois o cálculo do estêncil depende de pontos de células vizinhas que não podem ser sobrepostos. Desta forma, uma "nova" matriz de resultados é utilizada para armazenar os cálculos feitos a partir da matriz "antiga" que contém os dados. Quando a matriz "nova" está finalizada, a matriz "antiga" pode então ser descartada e utilizada para armazenar os resultados da próxima iteração, e desta forma as duas matrizes ficam alternando suas funções durante as iterações.

Ao término das iterações, é feita uma soma de todos os pontos da grade, para obter o total de calor.

Nó B710:

In [1]:
! lscpu | head -n 15 | grep "Model \|CPU(s):\|Thre\|Core\|NUMA\|MHz"

CPU(s):                24
Thread(s) per core:    1
Core(s) per socket:    12
NUMA node(s):          2
Model name:            Intel(R) Xeon(R) CPU E5-2695 v2 @ 2.40GHz
CPU MHz:               2435.302


## Fortran 90 sequencial

In [1]:
%%writefile heat_seq.f90
program stencil
    implicit none
    
    ! parameters for calculation
    integer             :: n=2400     ! n x n grid
    integer             :: energy=1   ! energy to be injected per iteration
    integer             :: niters=250 ! number of iterations

    ! other variables
    integer, parameter  :: nsources=3
    integer, dimension(3, 2)        :: sources
    double precision, allocatable   :: aold(:,:), anew(:,:)
    integer             :: iters, i, j, size, sizeStart, sizeEnd
    double precision    :: t=0.0, t1=0.0, heat=0.0           
    
    size = n + 2
    sizeStart = 2
    sizeEnd = n + 1

    allocate(aold(size, size))
    allocate(anew(size, size))
    aold = 0.0
    anew = 0.0

    sources(1,:) = (/ n/2,   n/2   /)
    sources(2,:) = (/ n/3,   n/3   /)
    sources(3,:) = (/ n*4/5, n*8/9 /)

    ! time measurement
    call cpu_time(t1)
    t = -t1

    do iters = 1, niters, 2
        
        ! odd iteration: anew <- stencil(aold)
            
        ! computationally intensive core
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                anew(i,j)=1/2.0*(aold(i,j)+  &
                    1/4.0*(aold(i-1,j)+aold(i+1,j)+ aold(i,j-1)+aold(i,j+1)))
            enddo
        enddo
        
        ! three-point energy insertion
        do i = 1, nsources
            anew(sources(i,1)+1, sources(i,2)+1) =  &
                anew(sources(i,1)+1, sources(i,2)+1) + energy
        enddo


        ! even iteration: aold <- stencil(anew)
           
        ! computationally intensive core
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                aold(i,j)=1/2.0*(anew(i,j)+  &
                    1/4.0*(anew(i-1,j)+anew(i+1,j)+anew(i,j-1)+anew(i,j+1)))
            enddo
        enddo
        
        ! three-point energy insertion       
        do i = 1, nsources
            aold(sources(i,1)+1, sources(i,2)+1) =  &
                aold(sources(i,1)+1, sources(i,2)+1) + energy
        enddo

    enddo

    ! sum of grid points to get total heat
    heat = 0.0
    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            heat = heat + aold(i,j)
        end do
    end do

    ! time measurement
    call cpu_time(t1)
    t = t + t1

    ! show de result
    write(*, "('Heat = ' f0.4' | ')", advance="no") heat
    write(*, "('Time = 'f0.4)") t

    deallocate(aold)
    deallocate(anew)

end

Writing heat_seq.f90


In [2]:
! gfortran --version

GNU Fortran (GCC) 4.8.5 20150623 (Red Hat 4.8.5-36)
Copyright (C) 2015 Free Software Foundation, Inc.

GNU Fortran comes with NO WARRANTY, to the extent permitted by law.
You may redistribute copies of GNU Fortran
under the terms of the GNU General Public License.
For more information about these matters, see the file named COPYING



In [3]:
! gfortran  -O3  -o heat_seq  heat_seq.f90

In [4]:
! ./heat_seq

Heat = 750.0000 | Time = 2.4168


> Nota: o nó de login não deve ser utilizado para rodar os programas finais; utilizá-lo apenas para compilar e fazer pequenos testes curtos.

## Python sequencial

In [5]:
! python --version

Python 3.8.5


In [6]:
import numpy as np
from time import time

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations

# other variables
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# computationally intensive core
def kernel(anew, aold) :
    anew[1:-1,1:-1]=1/2.0*(aold[1:-1,1:-1]+
            1/4.0*(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2]))

# main routine
t2 = 0
t0 = time()    # time measure
for _ in range(0, niters, 2) :
    t3 = time()
    kernel(anew, aold)    # odd iteration
    t2 += time() - t3
    anew[sources[:, 0], sources[:, 1]] += energy
    t3 = time()
    kernel(aold, anew)    # even iteration
    t2 += time() - t3
    aold[sources[:, 0], sources[:, 1]] += energy
heat[0] = np.sum( aold[1:-1, 1:-1] )  # system total heat
t1 = time()    # time measure

# show result
print("Heat: %0.4f | Time: %0.4f | Kernel: %0.4f" % (heat[0], t1-t0, t2) )

Heat: 750.0000 | Time: 24.3972 | Kernel: 24.3698


## Numba sequencial (1 thread)

In [4]:
import numpy as np
from time import time
from numba import njit, set_num_threads, get_num_threads, threading_layer

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations
# initialize the data arrays
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
# initialize three heat sources
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]
heat         = 0     # system total heat

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', fastmath=True, parallel=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1]=1/2.0*(aold[1:-1,1:-1]+
            1/4.0*(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2]))

# main routine
set_num_threads(1)
t2 = 0
t0 = time()    # time measure
for _ in range(0, niters, 2) :
    t3 = time()
    kernel(anew, aold)
    t2 += time() - t3
    anew[sources[:, 0], sources[:, 1]] += energy
    t3 = time()
    kernel(aold, anew)
    t2 += time() - t3
    aold[sources[:, 0], sources[:, 1]] += energy
heat = np.sum( aold[1:-1, 1:-1] )    # system total heat
t1   = time()    # time measure

# show result
print("Heat: %0.4f | Time: %0.4f | Kernel: %0.4f" % (heat, t1-t0, t2) )
print("Threading layer chosen: %s | Thread count: %s" %
      (threading_layer(), get_num_threads()) )

Heat: 750.0000 | Time: 3.3275 | Kernel: 3.2966
Threading layer chosen: tbb | Thread count: 1


## 4 threads

Obs: o hyperthreading e turboboost devem estar desligados, caso contrário não é possível ver o aumento de velocidade

In [2]:
import numpy as np
from time import time
from numba import njit, set_num_threads, get_num_threads, threading_layer

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations

# other variables
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', fastmath=True, parallel=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1]=1/2.0*(aold[1:-1,1:-1]+
            1/4.0*(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2]))

# main routine
set_num_threads(4)
t2 = 0
t0 = time()    # time measure
for _ in range((niters+1)//2) :
    t3 = time()
    kernel(anew, aold)
    t2 += time() - t3
    anew[sources[:, 0], sources[:, 1]] += energy
    t3 = time()
    kernel(aold, anew)
    t2 += time() - t3
    aold[sources[:, 0], sources[:, 1]] += energy
heat[0] = np.sum( aold[1:-1, 1:-1] )  # system total heat
t1 = time()    # time measure

# show result
print("Heat: %0.4f | Time: %0.4f | Kernel: %0.4f" % (heat[0], t1-t0, t2) )
print("Threading layer chosen: %s | Thread count: %s" %
      (threading_layer(), get_num_threads()) )

Heat: 750.0000 | Time: 1.2146 | Kernel: 1.1831
Threading layer chosen: tbb | Thread count: 4


## 16 threads

In [3]:
import numpy as np
from time import time
from numba import njit, set_num_threads, get_num_threads, threading_layer

# parameters
n            = 2400    # n x n grid
energy       = 1.0     # energy to be injected per iteration
niters       = 250     # number of iterations

# other variables
heat         = np.zeros((1), np.float64)     # system total heat
anew         = np.zeros((n + 2,  n + 2), np.float64)
aold         = np.zeros((n + 2,  n + 2), np.float64)
sources      = np.empty((3, 2), np.int16)    # sources of energy
sources[:,:] = [ [n//2, n//2], [n//3, n//3], [n*4//5, n*8//9] ]

# computationally intensive core
@njit('(float64[:,:],float64[:,:])', fastmath=True, parallel=True, nogil=True)
def kernel(anew, aold) :
    anew[1:-1,1:-1]=1/2.0*(aold[1:-1,1:-1]+
            1/4.0*(aold[2:,1:-1]+aold[:-2,1:-1]+aold[1:-1,2:]+aold[1:-1,:-2]))

# main routine
set_num_threads(16)
t2 = 0
t0 = time()    # time measure
for _ in range((niters+1)//2) :
    t3 = time()
    kernel(anew, aold)
    t2 += time() - t3
    anew[sources[:, 0], sources[:, 1]] += energy
    t3 = time()
    kernel(aold, anew)
    t2 += time() - t3
    aold[sources[:, 0], sources[:, 1]] += energy
heat[0] = np.sum( aold[1:-1, 1:-1] )  # system total heat
t1 = time()    # time measure

# show result
print("Heat: %0.4f | Time: %0.4f | Kernel: %0.4f" % (heat[0], t1-t0, t2) )
print("Threading layer chosen: %s | Thread count: %s" %
      (threading_layer(), get_num_threads()) )

Heat: 750.0000 | Time: 0.7777 | Kernel: 0.7479
Threading layer chosen: tbb | Thread count: 16
